# Uncorrecting Parallax

As part of KBMOD 2.0, we want the ability to take a set of images and “reproject” them into what we call “Explicit Barycentric Distance” (EBD) space, where we take a guess distance in AU from the sun (the “explicit” part in EBD) and reproject the image as if it was viewed from the Solar System’s barycenter. This change removes the parallax from the apparent motion of any moving objects and makes the path of the object appear more linear, and therefore more visible to KBMOD’s linear trajectory searching algorithm.

We have already done a lot of testing and validation of this approach on both simulated and real data, and we’ve already merged a lot of the needed code into the broader KBMOD project, and we’re pretty confident that further testing and improvements will result in better KBMOD search results. However, before reprojection becomes an integrated part of the KBMOD search pipeline, we need to address the issue of data provenance; if we find more TNOs through reprojected searching, but aren’t able to point in the original data where the object actually is, then we will have wasted a lot of compute and dev resources for nothing.

In the original KBMOD implementation, searching was done over pixel-space trajectories on the original images, and then when the results were filtered and clustered, we could easily pull the image stamps where we believed objects to be. In the new paradigm of pre facto barycentric reprojection, we run into a serious problem: the images that we’re searching for over have been resampled into a new frame during the search, but we’ll still need to be able to point to where in the original image the object is located before we can officially confirm that the object actually exists and was observed. This is further complicated by the fact that we are now combining data from different chips at the same obstime into single images.

To get back to original coordinates, let's try reversing the steps of parallax projection and test that against the original coordinate.

## Setup


In [1]:
from kbmod.reprojection_utils import correct_parallax

the ra, dec value here is what we want to get back to in the end.

In [2]:
from astropy.coordinates import EarthLocation, SkyCoord

ra, dec = 346.9681342111, -6.482196848597

time = "2021-08-24T20:59:06"
site = "ctio"
loc = EarthLocation.of_site(site)
distance = 41.1592725489203

now we'll actually run the parallax correction.

In [3]:
sc = SkyCoord(ra, dec, unit="deg")

corrected_coord = correct_parallax(sc, time, loc, distance)
corrected_coord

<ICRS Coordinate: (ra, dec, distance) in (deg, deg, AU)
    (346.64995304, -6.59350347, 41.15927254)>

## Inverse Parallax Correction

to get back to the original coordinate, we can just run backwards through the steps in [correct_parallax](https://github.com/dirac-institute/kbmod/blob/main/src/kbmod/reprojection_utils.py#L9).

In [4]:
import astropy.units as u
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, GCRS, ICRS
from astropy.wcs.utils import fit_wcs_from_points
from scipy.optimize import minimize

def invert_correct_parallax(coord, time, loc, distance):
    loc2 = (
        loc.x.to(u.m).value,
        loc.y.to(u.m).value,
        loc.z.to(u.m).value,
    ) * u.m
    x = sc.transform_to(GCRS(ra=sc.ra, dec=sc.dec, distance=distance, obstime=time, obsgeoloc=loc2))
    return x.transform_to(GCRS(obstime=time, obsgeoloc=loc2)).transform_to(ICRS())

In [5]:
fresh_ra, fresh_dec = corrected_coord.ra.degree, corrected_coord.dec.degree
fresh_ra, fresh_dec

(346.6499530448351, -6.5935034735937625)

Let's test: the smaller the difference between the original `SkyCoord` and `undone_sc`, than the better the result is.

In [6]:
fresh_sc = SkyCoord(ra=fresh_ra, dec=fresh_dec, unit="deg")

undone_sc = invert_correct_parallax(fresh_sc, time, loc, distance)
undone_sc

<SkyCoord (ICRS): (ra, dec) in deg
    (346.96813421, -6.48219685)>

In [7]:
sc.separation(undone_sc)

<Angle 2.70547099e-12 deg>

success!!